In [1]:
import pandas as pd
# gathering data from cryptocompare
# creating function to pull historical OHLCV data
# period is time period (day, hour, or minute), params is parameters, can be found at
# https://min-api.cryptocompare.com/
def get_historical_data(period, params, return_url=False):
    from urllib.parse import urlencode
    import requests
    
    # error messages
    p = ["day", "hour", "minute"]
    if(period.lower() not in p):
        print("Please use \"day\", \"hour\", or \"minute\".")
        return(-1)
    
    # other parameter errors
    # maximum for limit is 2000
    if(params['limit']>2000):
        print("Please use a limit <= 2000.")
        return(-1)
    
    data_url = 'https://min-api.cryptocompare.com/data/histo' + period.lower() + '?' + urlencode(params)
    
    data_json = requests.get(data_url).json()
    if(return_url):
        return(data_url)
    
    return(pd.DataFrame(data_json['Data']))

In [3]:
# getting minute, hourly, and daily data, with last data point always being 7/9/2018 00:00:00 (UTC)
import datetime as dt
date = '2018-07-09'
time = '00:00'
# need to specify UTC time zone, otherwise it uses local time zone
posix_dt = dt.datetime.strptime('{d} {t}'.format(d=date, t=time), '%Y-%m-%d %H:%M').replace(tzinfo=dt.timezone.utc).timestamp()

In [21]:
params = {'fsym' : 'XRP', 
         'tsym' : 'USD',
         'limit' : 2000,
         'toTs' : posix_dt}
df_minute = get_historical_data("minute", params)
df_day = get_historical_data("day", params)
df_hour = get_historical_data("hour", params)

In [22]:
# changing time for POSIX value to something more readable
df_minute['time'] = pd.to_datetime(df_minute['time'], unit='s')
df_day['time'] = pd.to_datetime(df_day['time'], unit='s')
df_hour['time'] = pd.to_datetime(df_hour['time'], unit='s')

In [23]:
file_name = params['fsym'] + '_to_' + params['tsym'] + '.csv'
times = ['minute', 'day', 'hour']
dfs = [df_minute, df_day, df_hour]
for i in range(3):
    file_name = params['fsym'] + '_to_' + params['tsym'] + times[i] + '.csv'
    dfs[i].to_csv(file_name)

## Making the Verification Dataset for Later

In [11]:
date = '2018-07-10'
time = '9:20'
time_now = dt.datetime.strptime('{d} {t}'.format(d=date, t=time), '%Y-%m-%d %H:%M').replace(tzinfo=dt.timezone.utc).timestamp()

In [12]:
params = {'fsym' : 'XRP', 
         'tsym' : 'USD',
         'limit' : 2000,
         'toTs' : time_now}
df = get_historical_data("minute", params)
df['time'] = pd.to_datetime(df['time'], unit='s')

In [14]:
df.tail()

,close,high,low,open,time,volumefrom,volumeto
1996,0.4586,0.4587,0.4586,0.4587,2018-07-10 09:16:00,5025.77,2306.52
1997,0.4586,0.4587,0.4585,0.4586,2018-07-10 09:17:00,30524.21,13998.72
1998,0.4585,0.4586,0.4585,0.4586,2018-07-10 09:18:00,4489.09,2057.09
1999,0.4575,0.4587,0.4575,0.4585,2018-07-10 09:19:00,555657.87,254146.65
2000,0.4561,0.4577,0.4561,0.4575,2018-07-10 09:20:00,219138.26,100049.19


In [15]:
file_name = 'XRP_to_USD_verfication.csv'
df.to_csv(file_name)